In [ ]:
ticker = "AMC"
TESTING = True

In [ ]:
API_KEY =  '0MXZO7Y5VXKK2R3PXN5XH8EZEWJTG6BJ' + '@AMER.OAUTHAP'
ACCT_NUMBER = '255612401'
CALLBACK_URL = 'https://localhost:8888'

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random

from tda import auth, client
from tda.orders.equities import equity_buy_market, equity_sell_market
from tda.orders.common import Duration, Session
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
from datetime import datetime
import json
import schedule
import time
from dateutil.relativedelta import relativedelta

sns.set()
%matplotlib inline

In [ ]:
pop_size = int(np.around(85.38952602372164))
learn_rate = max(min(0.287376193260779, 0.5), 0.000001)
sigma = max(min(0.22609652813829437, 1), 0.0001)
max_buy = 3
max_sell = 10
skip = int(np.around(1.3870273671208688))
window_size = int(np.around(24.88505443533264))
layer_size = int(np.around(695.5733189825535))
# Results of bayesian-agent run on 11/16 using 2 years of data
# {'learning_rate': 0.287376193260779, 'population_size': 85.38952602372164, 'sigma': 0.22609652813829437, 'size_network': 695.5733189825535, 
# 'skip': 1.3870273671208688, 'window_size': 24.88505443533264}

In [ ]:
import pkg_resources
import types

def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split('.')[0]
        elif isinstance(val, type):
            name = val.__module__.split('.')[0]
        poorly_named_packages = {'PIL': 'Pillow', 'sklearn': 'scikit-learn'}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        yield name


imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name != 'pip':
        requirements.append((m.project_name, m.version))

In [ ]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(layer_size, 1),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [ ]:
class TDAmeritrade:
    
    def __init__(self, account_number, api_key, callback_url, symbol):
        token_path = 'token.pickle'
        self.SYMBOL = symbol
        self.API_KEY = api_key
        self.CALLBACK_URL = callback_url
        self.ACCOUNT_NUMBER = account_number
        try:
            self.c = auth.client_from_token_file(token_path, self.API_KEY)
        except FileNotFoundError:
            from selenium import webdriver
            with webdriver.Chrome(ChromeDriverManager().install()) as driver:
                self.c = auth.client_from_login_flow(
                    driver, self.API_KEY, self.CALLBACK_URL, token_path)

        return
    
    def get_prices(self, end):
        # fetch price history using tda-api
        r = self.c.get_price_history(self.SYMBOL,
                                period_type=client.Client.PriceHistory.PeriodType.DAY,
                                period=client.Client.PriceHistory.Period.THREE_DAYS,
                                frequency_type=client.Client.PriceHistory.FrequencyType.MINUTE,
                                frequency=client.Client.PriceHistory.Frequency.EVERY_THIRTY_MINUTES,
                                end_datetime=end,
                                need_extended_hours_data=False
                                )

        assert r.status_code == 200, r.raise_for_status()

        # parse json and get candles data
        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
        df = pd.read_json(y)
        # drop last row
        df = df[:-1]

        return df
    
    def get_daily_price_history(self, symbol, start_date, end_date):
        r = self.c.get_price_history_every_day(symbol=symbol, start_datetime=start_date, end_datetime=end_date)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        y = y["candles"]
        y = json.dumps(y)
       
        df = pd.read_json(y)        
        return df

    def get_cur_price(self):
        r = self.c.get_quote(self.SYMBOL)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        price = y[self.SYMBOL]["lastPrice"]

        return price

    # get useful account info
    def get_account(self):
        r = self.c.get_account(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()

        return y

    # see if we currently have a position
    def get_available_funds(self, use_margin = False):
        y = self.get_account()
        if use_margin == True:
            return y['securitiesAccount']['currentBalances']['buyingPower']
        else:
            return y['securitiesAccount']['currentBalances']['availableFunds']
    
    def get_positions(self, symbol):
        pos = client.Client.Account.Fields.POSITIONS
        r = self.c.get_account(self.ACCOUNT_NUMBER, fields=[pos])
        y = r.json()
        if "positions" in y["securitiesAccount"]:
            y = y["securitiesAccount"]["positions"]    

            # Filter python objects where has instrument with symbol matchint ticker
            output_dict = [x for x in y if x['instrument']['symbol'] == ticker]
            y = json.dumps(output_dict)

            df = pd.read_json(y)
            return df[['averagePrice','longQuantity']] 
            
        else:
            return []
        

    def place_order(self, order_type, shares):
        if order_type == 'buy':
            order_spec = equity_buy_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)

        if order_type == 'sell':
            order_spec = equity_sell_market(self.SYMBOL, shares).set_session(
                Session.NORMAL).set_duration(Duration.DAY).build()
            self.c.place_order(self.ACCOUNT_NUMBER, order_spec)


    def get_transactions(self):
        r = self.c.get_transactions(self.ACCOUNT_NUMBER)
        assert r.status_code == 200, r.raise_for_status()

        y = r.json()
        
        return y


td_ameritrade = TDAmeritrade(ACCT_NUMBER, API_KEY, CALLBACK_URL, ticker)

In [ ]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [ ]:
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        historical_prices,
        model,
        money,
        max_buy,
        max_sell,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.historical_prices = historical_prices
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
    
        state = get_state(self.historical_prices, 0, self.window_size + 1)
        inventory = []
        quantity = 0
        l = len(self.historical_prices) - 1
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(self.historical_prices, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= self.historical_prices[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * self.historical_prices[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * self.historical_prices[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def trade(self, current_price, average_basis_of_inventory, quanity_of_ticker, current_inventory, days_since_training, historical_prices):
        state = get_state(historical_prices, days_since_training, self.window_size + 1)
        starting_money = self.initial_money
        states_sell = []
        states_buy = []
        quantity = current_inventory
        
        action, buy = self.act(state)
        if action == 1 and self.initial_money >= current_price:
            if buy < 0:
                buy = 1
            if buy > self.max_buy:
                buy_units = self.max_buy
            else:
                buy_units = buy

            if TESTING == False:
                print('WARNING!!!! SENDING REAL BUY ORDER')
                self.trading_api.place_order('buy', buy_units)

            total_buy = buy_units * current_price
            self.initial_money -= total_buy
            quantity += buy_units
            current_inventory += buy_units
            print('buy %d units at price %f, total balance %f' % (buy_units, current_price, self.initial_money) )

        elif action == 2 and quanity_of_ticker > 0:
            bought_price = average_basis_of_inventory
            print("quantity: " + str(quantity) + " self.max_sell: " + str(self.max_sell))
            if quantity > self.max_sell:
                sell_units = self.max_sell
            else:
                sell_units = quantity
            if sell_units < 1:
                return current_inventory

            if TESTING == False:
                print('WARNING!!!! SENDING REAL SELL ORDER')
                self.trading_api.place_order('sell', sell_units)

            current_inventory -= sell_units
            quantity -= sell_units
            total_sell = sell_units * average_basis_of_inventory
            self.initial_money += total_sell
            try:
                invest = ((total_sell - bought_price) / bought_price) * 100
            except:
                print('except')
                invest = 0
            print('sell %d units at price %f, investment %f %%, total balance %f,' % (sell_units, average_basis_of_inventory, invest, self.initial_money))
           
        
        invest = ((self.initial_money - starting_money) / starting_money) * 100
        return current_inventory

In [ ]:
current_date = datetime.today() - relativedelta(days=1)
past_date = current_date - relativedelta(years=2)
df = td_ameritrade.get_daily_price_history(ticker, past_date, current_date)
df.rename(columns = {'datetime':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)

In [ ]:
open = df.Open.values.tolist()

avail_funds = td_ameritrade.get_available_funds()
print("Available Funds: "  + str(avail_funds))

model = Model(input_size = window_size, layer_size = layer_size, output_size = 3)
agent = Agent(
              population_size=pop_size,
              learning_rate = learn_rate,
              historical_prices=open,
              sigma=sigma,
              money=avail_funds,
              max_buy=max_buy,
              max_sell=max_sell,
              model = model, 
              window_size = window_size,
              skip = skip)
              
agent.fit(iterations = 100, checkpoint = 5)

In [ ]:
avail_funds = td_ameritrade.get_available_funds(False)
print("Available Funds: "  + str(avail_funds))

try:
    positions = td_ameritrade.get_positions(ticker) 
    inventory = positions.longQuantity.values[0]
    val = float(positions.averagePrice.values[0])
except:
    inventory = 0
    val = 0

current_date = datetime.today() #- relativedelta(days=1)
past_date = current_date - relativedelta(years=2)
df = td_ameritrade.get_daily_price_history(ticker, past_date, current_date)
df.rename(columns = {'datetime':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)

last_dt = df.iloc[-1]['Date']
now_dt = datetime.now()

countdown = now_dt - last_dt
days_since_training_the_model = countdown.days
print(days_since_training_the_model)

try:
    positions = td_ameritrade.get_positions(ticker)
    qty = positions.longQuantity.values[0]
    avg_price = positions.averagePrice.values[0]
    cur_price = td_ameritrade.get_cur_price()    
except:
    qty = 0 
    avg_price = 0
    cur_price = td_ameritrade.get_cur_price()

print("Price: " + str(cur_price) + " Day: " + str(days_since_training_the_model) + "Current Inventory: " + str(inventory))    
open = df.Open.values.tolist()
inv =  agent.trade(cur_price, avg_price, qty, inventory, days_since_training_the_model, open)
        
price = td_ameritrade.get_cur_price()
value = price * inventory
print('New Inventory: ' + str(inventory))
print('Current Value of Inventory: ' + str(value))